### Lab Task 1 - Dataset Preparation, Feature Scaling

In [33]:
import sklearn.datasets
import pandas as pd
import numpy as np

In [3]:
data = sklearn.datasets.fetch_california_housing()
print(data)


{'data': array([[   8.3252    ,   41.        ,    6.98412698, ...,    2.55555556,
          37.88      , -122.23      ],
       [   8.3014    ,   21.        ,    6.23813708, ...,    2.10984183,
          37.86      , -122.22      ],
       [   7.2574    ,   52.        ,    8.28813559, ...,    2.80225989,
          37.85      , -122.24      ],
       ...,
       [   1.7       ,   17.        ,    5.20554273, ...,    2.3256351 ,
          39.43      , -121.22      ],
       [   1.8672    ,   18.        ,    5.32951289, ...,    2.12320917,
          39.43      , -121.32      ],
       [   2.3886    ,   16.        ,    5.25471698, ...,    2.61698113,
          39.37      , -121.24      ]], shape=(20640, 8)), 'target': array([4.526, 3.585, 3.521, ..., 0.923, 0.847, 0.894], shape=(20640,)), 'frame': None, 'target_names': ['MedHouseVal'], 'feature_names': ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'], 'DESCR': '.. _california_housing_dataset

In [8]:
data.keys()

dict_keys(['data', 'target', 'frame', 'target_names', 'feature_names', 'DESCR'])

In [16]:
data['feature_names']

['MedInc',
 'HouseAge',
 'AveRooms',
 'AveBedrms',
 'Population',
 'AveOccup',
 'Latitude',
 'Longitude']

In [17]:
data['target_names']

['MedHouseVal']

In [20]:
new_data={}
new_data['data']=data['data'][:,[0,1,2]]
new_data['feature_names']=data['feature_names'][:3]
new_data['feature_names']

['MedInc', 'HouseAge', 'AveRooms']

In [22]:
new_data['target']=data['target']
new_data['target_names']=data['target_names']
new_data['target_names']

['MedHouseVal']

In [29]:
# Splitting dataset randomly
data['data'].shape

(20640, 8)

In [37]:

# create and shuffle a list of indices
idx_list=[i for i in range(len(new_data['data']))]
shuffled_idx_list=np.random.permutation(np.array(idx_list))

# split the indices list into train and val indices
_80thidx=(0.8*len(new_data['data']))
train_indices_list=shuffled_idx_list[:int(_80thidx)]
val_indices_list=shuffled_idx_list[int(_80thidx):]

print(len(train_indices_list)+len(val_indices_list))

[0, 1, 2, 3, 4]
[18872  2691  8354 20278  4355]


In [50]:
# splitting  train data
train_data={}
train_data['data']=new_data['data'][train_indices_list]
train_data['target']=new_data['target'][train_indices_list]
train_data['feature_names']=new_data['feature_names']
train_data['target_names']=new_data['target_names']
train_data['data'].shape

In [62]:
# splitting val data
val_data={}
val_data['data']=new_data['data'][val_indices_list]
val_data['target']=new_data['target'][val_indices_list]
val_data['feature_names']=new_data['feature_names']
val_data['target_names']=new_data['target_names']
val_data['data'].shape

(4128, 3)

In [63]:
# converting to a Dataframe and then to csv

# train data
train_df=pd.DataFrame(train_data['data'],columns=train_data['feature_names'])
train_df['target']=train_data['target']
print(train_df[:3])


val_df=pd.DataFrame(val_data['data'],columns=val_data['feature_names'])
val_df['target']=val_data['target']
print(val_df[:3])


train_df.to_csv('train_data.csv',index=False)
val_df.to_csv('val_data.csv',index=False)

print("data saved in train_data.csv and val_data.csv")

   MedInc  HouseAge  AveRooms  target
0  5.2957      18.0  6.483932   1.719
1  1.9911      10.0  5.575668   0.675
2  5.0912      44.0  6.457576   1.698


### Lab Task 2 - Cost Function without and with Regularization

In [71]:
import random
def cost_function(features,target,lambd):
    w=np.random.rand(1,1+features.shape[1])
    h_x=
    
    print(w.shape)
    
    
    return None

In [72]:
cost_function(train_data['data'],0,0)

(1, 4)


In [74]:
type(train_data['data'])

numpy.ndarray